# 提取所有题目信息

In [11]:
import json
import pandas as pd

In [12]:
LANGUAGE_LIST = ["python", "javascript", "typescript", "java", "c&cpp"]
SUFFIX_MAP = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "c&cpp": "cpp",
    "java": "java"
}


def generate_python_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a python function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a python class , the class signature as below {code_signature}"


def generate_javascript_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a javascript function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a javascript class , the class signature as below {code_signature}"


def generate_typescript_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a typescript function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a typescript class , the class signature as below {code_signature}"


def generate_ccpp_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a cpp function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a cpp class , the class signature as below {code_signature}"


def generate_java_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a java function , the function signature as below {code_signature},package is org.real.temp, class is Answer"
    elif code_type == "class":
        return f"please write a java class , the function signature as below {code_signature},package is org.real.temp, class is Answer"


LANGUAGE_PROMPT_MAP = {
    "python": generate_python_prompt,
    "javascript": generate_javascript_prompt,
    "typescript": generate_typescript_prompt,
    "c&cpp": generate_ccpp_prompt,
    "java": generate_java_prompt
}

In [17]:
total_question = []
need_question_list = ["python","c&cpp","java","javascript","typescript"]
for language in LANGUAGE_LIST:
    excel_data = pd.read_excel("../xlsx/RealisticEval-Data.xlsx", sheet_name=language)
    data = excel_data[excel_data['check'] == 'yes']
    # 遍历每一行
    for index, row in data.iterrows():
        task_id = int(row['task_id'])
        code_type = row['code_type']
        original_language = language
        question_type = row['question_type']
        summary = row['task_summary_en']
        temp_data = {
            "task_id": task_id,
            "code_type": code_type,
            "original_language": original_language,
            "question_type": question_type,
            "summary": summary, "language_version_list": {
                "python": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                },
                "javascript": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                },
                "typescript": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                },
                "c&cpp": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                },
                "java": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                }
            }
        }
        for q in need_question_list:
            try:
                if language == q:
                    pass
                elif row[q] != "finish":
                    continue
                language_version_list = temp_data["language_version_list"]
                language_info = language_version_list[q]
                dir_path = f"../../all/t{task_id}"
                if q == "java":
                    signature_path = f"{dir_path}/{q}/Signature.{SUFFIX_MAP[q]}"
                else:
                    signature_path = f"{dir_path}/{q}/signature.{SUFFIX_MAP[q]}"
                addition_info = str(row[f"{q}_addition_info"])
                if addition_info == 'nan':
                    addition_info = ""
                with open(signature_path, "r", encoding="utf8") as signature_file:
                    code_signature = signature_file.read()
                if q == "java":
                    test_path = f"{dir_path}/{q}/Tester.{SUFFIX_MAP[q]}"
                else:
                    test_path = f"{dir_path}/{q}/test.{SUFFIX_MAP[q]}"
                with open(test_path, "r", encoding="utf8") as test_file:
                    code_test = test_file.read()
                prompt = LANGUAGE_PROMPT_MAP[q](code_signature, code_type)
                language_info["code_signature"] = code_signature
                language_info["test_code"] = code_test
                language_info["prompt"] = prompt
                language_info["addition_info"] = addition_info
            except Exception as e:
                print(e)
                continue
        with open(f"../question/RealisticCodeBench_{language}.jsonl","a+",encoding="utf8") as file:
            json_str=json.dumps(temp_data)
            file.write(json_str+"\n")
            file.flush()

[Errno 2] No such file or directory: '../../all/t434/c&cpp/signature.cpp'
[Errno 2] No such file or directory: '../../all/t434/java/Signature.java'
[Errno 2] No such file or directory: '../../all/t434/javascript/signature.js'
[Errno 2] No such file or directory: '../../all/t434/typescript/signature.ts'
[Errno 2] No such file or directory: '../../all/t461/c&cpp/signature.cpp'
[Errno 2] No such file or directory: '../../all/t461/java/Signature.java'
[Errno 2] No such file or directory: '../../all/t461/typescript/signature.ts'
[Errno 2] No such file or directory: '../../all/t618/python/signature.py'
[Errno 2] No such file or directory: '../../all/t618/c&cpp/signature.cpp'
[Errno 2] No such file or directory: '../../all/t618/java/Signature.java'


# 单语言问题信息提取

In [ ]:
for language in LANGUAGE_LIST:
    # 读取excel表 获取数据信息
    excel_data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx", sheet_name=language)
    data = excel_data[excel_data['check'] == 'yes']
    # 遍历每一行
    question_array = []

    for index, row in data.iterrows():
        task_id = int(row["task_id"])
        code_type = row["code_type"]
        dir_path = f"../all/t{task_id}"
        if language == "java":
            signature_path = f"{dir_path}/{language}/Signature.{SUFFIX_MAP[language]}"
        else:
            signature_path = f"{dir_path}/{language}/signature.{SUFFIX_MAP[language]}"
        addition_info = str(row["addition_info"])
        if addition_info == 'nan':
            addition_info = ""
        with open(signature_path, "r", encoding="utf8") as signature_file:
            code_signature = signature_file.read()
        if language == "java":
            test_path = f"{dir_path}/{language}/Tester.{SUFFIX_MAP[language]}"
        else:
            test_path = f"{dir_path}/{language}/test.{SUFFIX_MAP[language]}"
        with open(test_path, "r", encoding="utf8") as test_file:
            code_test = test_file.read()
        prompt = LANGUAGE_PROMPT_MAP[language](code_signature, code_type)
        question_info = {
            "task_id": task_id,
            "code_type": code_type,
            "code_signature": code_signature,
            "code_language": language,
            "test_code": code_test,
            "prompt": prompt,
            "addition_info": addition_info

        }
        question_array.append(question_info)
        print(f"task_id:{task_id} signature:{len(code_signature)} test:{len(code_test)}")
    with open(f"./question/{language}.json", "w", encoding="utf8") as question_file:
        json_str = json.dumps(question_array)
        question_file.write(json_str)
        question_file.flush()
    with open(rf"E:\code\code_back\python_project\llm\qa\question\{language}.json", "w",encoding="utf8") as question_file:
        json_str = json.dumps(question_array)
        question_file.write(json_str)
        question_file.flush()

# 数据集问题类型统计-代码类型

In [ ]:
import pandas as pd

In [ ]:
language_list = ["python","javascript","typescript","java","c&cpp"]
class_number = 0
func_number = 0
for language in language_list:
    data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx",sheet_name=language)
    method_data = data[data['code_type']=='method']
    class_data = data[data['code_type'] == 'class']
    func_number = func_number + len(method_data)
    class_number =class_number +len(class_data)
print("class number:",class_number)
print("func number:",func_number)

# 数据集问题统计-不同类型代码问题集合

In [ ]:
import pandas as pd

In [ ]:
language_list = ["python","javascript","typescript","java","c&cpp"]
result = {
    "python":0,
    "javascript":0,
    "typescript":0,
    "c&cpp":0,
    "java":0
}
for language in language_list:
    data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx",sheet_name=language)
    count1 = data['check'].value_counts().get("yes",0)
    result[language] = result[language]+count1
    for l in language_list:
        if l == language:
            continue
        count2 = data[l].value_counts().get('finish',0)
        result[l] = result[l]+count2
result

# 根据question_type 统计数量

In [ ]:
import pandas as pd

In [ ]:
language_list = ["python","javascript","typescript","java","c&cpp"]
result_dict_list = []
for language in language_list:
    data = pd.read_excel("../xlsx/RealisticEval-Data.xlsx",sheet_name=language)
    series = data['new_question_type'].value_counts()
    result_dict_list.append(series.to_dict())
result = {}
for d in result_dict_list:
    for key, value in d.items():
        # 如果键已存在，则累加其值；否则，初始化该键的值
        if key in result:
            result[key] += value
        else:
            result[key] = value
result

# 根据input type 统计参数数量

In [ ]:
import pandas as pd

In [ ]:
language_list = ["python","javascript","typescript","java","c&cpp"]
result_dict_list = []
for language in language_list:
    data = pd.read_excel("../xlsx/RealisticEval-Data.xlsx",sheet_name=language)
    series = data['input_type'].value_counts()
    result_dict_list.append(series.to_dict())
result = {}
for d in result_dict_list:
    for key, value in d.items():
        # 如果键已存在，则累加其值；否则，初始化该键的值
        if key in result:
            result[key] += value
        else:
            result[key] = value
final_result = {}
for key,value in result.items():
    key_list = key.split(",")
    if len(key_list) == 1:
        is_ok = final_result.get(key_list[0],None)
        if is_ok:
            final_result[key_list[0]] = final_result[key_list[0]]+value
        else:
            final_result[key_list[0]] = value
    else:
        for k in key_list:
            is_ok = final_result.get(key_list[0],None)
            if is_ok:
                final_result[key_list[0]] = final_result[key_list[0]]+value
            else:
                final_result[key_list[0]] = value
sorted_dict = dict(sorted(final_result.items(), key=lambda item: item[1],reverse=True))
sorted_dict

# 统计回答问题行数

In [ ]:
import json
import pandas

In [ ]:
language_list = ['python','javascript','typescript','java','c&cpp']
SUFFIX_MAP = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "c&cpp": "cpp",
    "java": "java"
}
answer_file_line_result = {
    "python":[],
    "java":[],
    "javascript":[],
    "typescript":[],
    "c&cpp":[]
}

with open("../question/all_question.json","r",encoding="utf8") as file:
    data_list  = json.load(file)
for item in data_list:
    try:
        task_id = item['task_id']
        question_dir_path = rf"E:\code\code_back\python_project\RealisticEval-Data\all\t{task_id}"
        for language in language_list:
            language_item = item['language_version_list'][language]
            if language_item['code_signature'] =="":
                continue
            if language == "java":
                answer_file = "Answer.java"
            else:
                answer_file = f"answer.{SUFFIX_MAP[language]}"
            language_answer_path = rf"{question_dir_path}\{language}\{answer_file}"
            with open(language_answer_path,'r',encoding="utf8") as file:
                answer_lines = len(file.readlines())
                if answer_lines == 0:
                    continue
                answer_file_line_result[language].append(answer_lines)
    except Exception as e:
        print(e)
        continue
all_language_average = 0
for key,value in answer_file_line_result.items():
    print(f"{key},line:{sum(value)/len(value)}")
    all_language_average  = all_language_average+sum(value)/len(value)
print("all language average",all_language_average/5)

# 计算prompt token 数

In [ ]:
import json

In [ ]:
language_list = ['python','javascript','typescript','java','c&cpp']
SUFFIX_MAP = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "c&cpp": "cpp",
    "java": "java"
}
answer_file_line_result = {
    "python":[],
    "java":[],
    "javascript":[],
    "typescript":[],
    "c&cpp":[]
}
def estimate_token_count(input_string):
    # 将字符串按空格分割
    words = input_string.split()
    # 通常每个词会被视为一个 token，添加额外的 token 用于标点符号和其他元素
    token_count = len(words) + sum(1 for word in words if any(char in word for char in ['.', ',', '!', '?', ';', ':']))
    return token_count
with open("../question/all_question.json","r",encoding="utf8") as file:
    data_list  = json.load(file)
for item in data_list:
    try:
        task_id = item['task_id']
        question_dir_path = rf"E:\code\code_back\python_project\RealisticEval-Data\all\t{task_id}"
        for language in language_list:
            language_item = item['language_version_list'][language]
            prompt = language_item['prompt']
            if prompt == "":
                continue
            prompt_count = estimate_token_count(prompt)
            answer_file_line_result[language].append(prompt_count)
    except Exception as e:
        print(e)
        continue
all_language_average = 0
for key,value in answer_file_line_result.items():
    print(f"{key} tokens:{sum(value)/len(value)}")
    all_language_average  = all_language_average+sum(value)/len(value)
print("all language average",all_language_average/5)

# 计算距离

In [ ]:
import json
import pandas as pd
import re

class AdditionInfoRemove:

    def remove(self,language, code):
        if language == "python":
            return self.python_remove(code)
        elif language == "javascript":
            return self.javascript_remove(code)
        elif language == "typescript":
            return self.typescript_remove(code)
        elif language == "java":
            return self.java_remove(code)
        elif language == "c&cpp":
            return self.cpp_remove(code)
    def python_remove(self,code):
        # 移除单行注释
        code = re.sub(r'#.*', '', code)

        # 移除多行注释
        code = re.sub(r'"""(.*?)"""', '', code, flags=re.DOTALL)
        code = re.sub(r"'''(.*?)'''", '', code, flags=re.DOTALL)

        # 移除空行
        code = re.sub(r'\n\s*\n', '\n', code)  # 移除空行
        code = code.strip()  # 去掉开头和结尾的空行

        return code

    def javascript_remove(self,code):
        js_code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
        # 移除单行注释（// ...）
        js_code = re.sub(r'//.*?$', '', js_code, flags=re.MULTILINE)
        js_code = re.sub(r'\n\s*\n', '\n', js_code)  # 移除空行

        return js_code

    def typescript_remove(self,code):
        js_code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
        # 移除单行注释（// ...）
        js_code = re.sub(r'//.*?$', '', js_code, flags=re.MULTILINE)
        js_code = re.sub(r'\n\s*\n', '\n', js_code)  # 移除空行

        return js_code

    def java_remove(self,code):
        # 移除多行注释（/* ... */）
        java_code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
        # 移除单行注释（// ...）
        java_code = re.sub(r'//.*?$', '', java_code, flags=re.MULTILINE)
        java_code = re.sub(r'\n\s*\n', '\n', java_code)  # 移除空行
        return java_code

    def cpp_remove(self,code):
        # 移除多行注释（/* ... */）
        cpp_code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
        # 移除单行注释（// ...）
        cpp_code = re.sub(r'//.*?$', '', cpp_code, flags=re.MULTILINE)
        cpp_code = re.sub(r'\n\s*\n', '\n', cpp_code)  # 移除空行
        return cpp_code


def normal_leven(str1, str2):
  len_str1 = len(str1) + 1
  len_str2 = len(str2) + 1
  # 创建矩阵
  matrix = [0 for n in range(len_str1 * len_str2)]
  #矩阵的第一行
  for i in range(len_str1):
    matrix[i] = i
  # 矩阵的第一列
  for j in range(0, len(matrix), len_str1):
    if j % len_str1 == 0:
      matrix[j] = j // len_str1
  # 根据状态转移方程逐步得到编辑距离
  for i in range(1, len_str1):
    for j in range(1, len_str2):
      if str1[i-1] == str2[j-1]:
        cost = 0
      else:
        cost = 1
      matrix[j*len_str1+i] = min(matrix[(j-1)*len_str1+i]+1,
                    matrix[j*len_str1+(i-1)]+1,
                    matrix[(j-1)*len_str1+(i-1)] + cost)

  return matrix[-1]  # 返回矩阵的最后一个值，也就是编辑距离

In [ ]:
language_list = ['python',"javascript","typescript","c&cpp","java"]
SUFFIX_MAP = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "c&cpp": "cpp",
    "java": "java"
}
levenshtein_distance_result = {
    "python":[],
    "javascript":[],
    "typescript":[],
    "c&cpp":[],
    "java":[]
}
for language in language_list:
    print(language)
    data_list =[]
    # 读取回答信息
    with open(rf"E:\code\code_back\python_project\llm\qa\gpt-4_answer\{language}_answer_pass1.jsonl","r",encoding="utf8") as file:
        json_lines = [line.strip() for line in file.readlines()]
        for item in json_lines:
            data_list.append(json.loads(item))
    # map 散列
    data_list_id_map = {}
    for item in data_list:
        data_list_id_map[item['task_id']] = item
    # 多余内容去除
    remover = AdditionInfoRemove()
    # 读取文件数据
    data = pd.read_excel("../xlsx/RealisticEval-Data.xlsx",sheet_name=language)
    for index,row in data.iterrows():
        try:
            task_id = row['task_id']
            question_dir_path = rf"E:\code\code_back\python_project\RealisticEval-Data\all\t{task_id}"
            original_file_path = rf"{question_dir_path}\original.{SUFFIX_MAP[language]}"
            with open(original_file_path,"r",encoding="utf8") as file:
                original_code = file.read()
            original_code = remover.remove(language,original_code)
            # 找到对应问题的
            data_item = data_list_id_map[task_id]
            answer_code = data_item['language_version_list'][language]['answer_list'][0]['response_code']
            answer_code = remover.remove(language,answer_code)
            distance = normal_leven(original_code,answer_code)
            levenshtein_distance_result[language].append(distance)
        except Exception as e:
            print(e)
            continue
        # for key,value in levenshtein_distance_result.items():
        #     if len(value) == 0:
        #         continue
        #     print(f"{key} distance:{sum(value)/len(value)}")
for key,value in levenshtein_distance_result.items():
    if len(value) == 0:
        continue
    print(f"{key} distance:{sum(value)/len(value)}")